In [1]:
import cobra
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

In [ ]:
# create dictionary for all models
models = {}

# load human GEM into the dictionary
models["ihuman"] = cobra.io.load_matlab_model("./models/Human-GEM.mat")

In [ ]:
# load ftINIT models into the dictionary
directory = os.fsencode("./models")
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("_model.mat"): 
        pos = filename.find("_")
        models[filename[:pos]] = cobra.io.load_matlab_model("./models/"+filename)
    else:
        continue

print(models)

In [ ]:
# Storing pathways in variable

pathways = []
for pathway in models["ihuman"].groups:
    pathways.append(pathway.name)
print(pathways)
print(len(pathways))

# correct pathway 1 for labelling figures
pathways_corr = pathways
pathways_corr[0] = pathways[0][1:]
pathways_corr[147] = "Octane oxidation"
print(pathways_corr)

In [ ]:
# analyzing differences in existing pathways between models

model_pathways = {}
for model in models:
    pathwaylist = []
    for pathway in models[model].groups:
        pathwaylist.append(pathway.name)
    model_pathways[model] = pathwaylist
    missing_pathways = set(pathways)-set(model_pathways[model])
    print(f"{model}: {len(models[model].groups)} (missing: {missing_pathways})")
print()
print(model_pathways)

In [ ]:
# Creating an overview of existing pathways

import pandas as pd

# Creating a dictionary for boolean description of pathway existence
pathway_exist = {}
for model in models.keys():
    pathway_exist[model] = []

for pathway in pathways:
    for model in models.keys():
        if pathway in model_pathways[model]:
            pathway_exist[model].append(0)
        else: 
            pathway_exist[model].append(1)

indices = model_pathways["ihuman"]
print(indices)
pathway_exist_df = pd.DataFrame(pathway_exist, index=model_pathways["ihuman"])
print(pathway_exist_df)
pathway_exist_df.to_csv("./outputs/ftINIT_models_existing_pathways.tsv", sep="\t")

In [ ]:
# Defining a function to draw heatmaps

def heatmap(data, x_label, y_label, factor_label: str, x_fontsize: int=10, y_fontsize: int=8, fontfamily="Bahnschrift", colortype: str='plasma', shrinkfct: float=0.1, figuresize: tuple=(22, 22), symmetric_scale: bool=False, abs_max=None, save_as: str=None):
    """
    colortype: Colorscale of the colormap
        Perceptually Uniform Sequential:
        - viridis (default)
        - plasma
        - inferno
        - magma
        - cividis (colorblind-friendly)
        Non-Uniform Sequential:
        - Greys
        - Purples
        - Blues
        - Oranges
        - Reds
        - Greens
        Diverging:
        - coolwarm
        - seismic
        - Spectral
        - PiYG (Pink-Green)
        - PRGn (Purple-Green)
        - RdBu (Red-Blue)
        - RdGy (Red-Grey)
        - BrBG (Brown-BlueGreen)
        Qualitative:
        - tab10 (default for discrete categories)
        - Set1, Set2, Set3
        - Accent
        - Dark2
        - Paired
        - Pastel1, Pastel2
        Cyclic:
        - twilight
        - twilight_shifted
        - hsv
        Miscellaneous Colormaps
        - jet (legacy, not recommended for scientific work due to perceptual issues)
        - rainbow (colorful but not perceptually uniform)
        - flag
        - gist_rainbow
        A _r following a colortype inverts the colors
    """

    # Determine the normalization for the colormap
    if symmetric_scale:
        if abs_max is None:
            abs_max = max(abs(data.min()), abs(data.max()))  # Use the maximum absolute value in the data
        norm = Normalize(vmin=-abs_max, vmax=abs_max)  # Symmetric normalization
    else:
        norm = None  # Default normalization

    # Plot the heatmap
    plt.figure(figsize=figuresize)  # Adjust figure size for square heat fields
    ax = plt.gca()  # Get the current axis
    heatmap = ax.imshow(data, cmap=colortype, aspect='equal', interpolation='nearest', norm=norm)  # aspect='equal' ensures square fields

    # Add color bar with adjusted thickness
    cbar = plt.colorbar(heatmap, shrink=shrinkfct)  # Shrink reduces the size of the color bar
    cbar.set_label(factor_label, rotation=270, labelpad=20)

    # Set axis labels
    ax.set_xticks(np.arange(len(x_label)))
    ax.set_xticklabels(x_label, rotation=90, fontsize=x_fontsize, fontfamily=fontfamily)  # Smaller font size for x labels
    ax.xaxis.tick_top()  # Move x-axis labels to the top

    # Show all y-axis labels with smaller font size
    ax.set_yticks(np.arange(len(y_label)))
    ax.set_yticklabels(y_label, fontsize=y_fontsize, fontfamily=fontfamily)  # Reduced font size for y labels

    # Remove the title
    plt.tight_layout()

    # Save he plot
    if save_as != None:
        plt.savefig(save_as, dpi=300, transparent=True, bbox_inches='tight')

    # Show the plot
    plt.show()

In [ ]:
# plotting pathway existence as a heatmap
heatmap(pathway_exist_df.to_numpy(), models.keys(), pathways_corr, "exist", colortype="seismic", symmetric_scale=True, save_as="./outputs/ftINIT_models_pathway_exist.png")

# Comparing the fluxes of different Pathways

In [ ]:
# performing FBA on all models
FBA_solutions = {}
for model in models:
    FBA_solutions[model] = models[model].optimize()

In [ ]:
# creating an empty dictionary of lists for the flux values
pathway_fluxes = {}
for model in models.keys():
    pathway_fluxes[model] = []


# filling in the flux values for all reactions
for model in models:
    for pathway in models["ihuman"].groups:
        pathway_flux = 0
        for rxn in pathway.members:
            if rxn in models[model].reactions:
                pathway_flux += FBA_solutions[model].fluxes[rxn.id]
        pathway_fluxes[model].append(pathway_flux)

# creating a pandas dataframe out of the dictionary
pathway_fluxes_df = pd.DataFrame(pathway_fluxes, index=pathways_corr)
pathway_fluxes_df.to_csv("./outputs/ftINIT_models_pathway_fluxes.tsv", sep="\t")
pathway_fluxes_df

In [ ]:
# visualizing the fluxes as a heatmap
heatmap(pathway_fluxes_df.to_numpy(), models.keys(), pathways_corr, 'sum of fluxes through the pathway', symmetric_scale=True, colortype="seismic", save_as="./outputs/ftINIT_models_pathway_abs_flux.png")

In [ ]:
# differences in smaller values are hard to see in the heatmap above

# transforming the data so that values over 10 and the absolutes of the ones lower than -10 are logarithmized, 
# while the values between -10 and 10 remain linear normalized to the range between 1- and 1
pathway_fluxes_poslog = np.where(pathway_fluxes_df.to_numpy() > 10, np.log10(abs(pathway_fluxes_df.to_numpy())), pathway_fluxes_df.to_numpy())
pathway_fluxes_neglog = np.where(pathway_fluxes_df.to_numpy() < -10, ((-1)*(np.log10(abs(pathway_fluxes_df.to_numpy())))), pathway_fluxes_poslog)
pathway_fluxes_symlog = np.where((pathway_fluxes_df.to_numpy() >= -10) & (pathway_fluxes_df.to_numpy() <= 10), (pathway_fluxes_df.to_numpy()/10), pathway_fluxes_neglog)

# visualizing transformed data as a heatmap
heatmap(pathway_fluxes_symlog, models.keys(), pathways_corr, 'symlog of the sum of fluxes through the pathway', symmetric_scale=True, colortype="seismic", save_as="./outputs/ftINIT_models_pathway_log_flux.png")

In [ ]:
# making a heatmap of flux changes to the human model
pathway_fluxes_np = pathway_fluxes_df.to_numpy()
pathway_flux_dif = (pathway_fluxes_np - pathway_fluxes_np[:,[0]])

heatmap(pathway_flux_dif, models.keys(), pathways_corr, 'difference of the sum of fluxes through the pathway to the ihuman model', symmetric_scale=True, colortype="seismic", save_as="./outputs/ftINIT_models_pathway_differences.png")

# Calculating individual fluxes

In [ ]:
# creating an empty dictionary of lists for the flux values
reaction_fluxes = {}
for model in models.keys():
    reaction_fluxes[model] = []

rxns = models["ihuman"].reactions
rxn_ids = {}
for rxn in rxns:
    rxn_ids[rxn.id] = rxn

# filling in the flux values for all reactions
for id in rxn_ids:
    for model in models:
        if rxn_ids[id] in models[model].reactions:
            reaction_fluxes[model].append(FBA_solutions[model].fluxes[id])
        else:
            reaction_fluxes[model].append(0)


# creating a pandas dataframe out of the dictionary
reaction_fluxes_df = pd.DataFrame(reaction_fluxes, index=rxn_ids.keys())
reaction_fluxes_df.to_csv("./outputs/ftINIT_models_reaction_fluxes.tsv", sep="\t")
reaction_fluxes_df

# Clustering

In [ ]:
# Transpose the data so models are rows
flux_data_T = reaction_fluxes_df.T

In [ ]:
# Perform clustering on models
from sklearn.cluster import KMeans

n_clusters = 5  # Random number of clusters 
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
model_clusters = kmeans.fit_predict(flux_data_T)

In [ ]:
# visualizing the clusters
from sklearn.decomposition import PCA

# Perform PCA for dimensionality reduction
pca = PCA(n_components=2)
models_pca = pca.fit_transform(flux_data_T)

# Scatter plot with cluster labels
plt.figure(figsize=(10, 8))

# Plot each model as a point
for i, label in enumerate(flux_data_T.index):  # Use the transposed data's index (model names)
    plt.scatter(models_pca[i, 0], models_pca[i, 1], c=f'C{model_clusters[i]}', s=50, label=f'Cluster {model_clusters[i]}' if i == 0 else None)
    plt.text(models_pca[i, 0] + 0.02, models_pca[i, 1] + 0.02, label, fontsize=8, alpha=0.7)

# Plot decorations
plt.title('Model Clustering Visualization', fontsize=14)
plt.xlabel('PCA Component 1', fontsize=12)
plt.ylabel('PCA Component 2', fontsize=12)
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# Get loadings (components)
loadings = pd.DataFrame(pca.components_.T, index=reaction_fluxes_df.index, columns=[f'PC{i+1}' for i in range(pca.n_components_)])

print(f"PC1 explains {pca.explained_variance_ratio_[0]} of the variance.")
print(f"Reaction {loadings["PC1"].idxmax()} has the biggest impact on the component ({loadings["PC1"].max()})")
print(models["ihuman"].reactions.get_by_id(loadings["PC1"].idxmax()).name)
print()
print(f"PC2 explains {pca.explained_variance_ratio_[1]} of the variance.")
print(f"Reaction {loadings["PC2"].idxmax()} has the biggest impact on the component ({loadings["PC2"].max()})")
print(models["ihuman"].reactions.get_by_id(loadings["PC2"].idxmax()).reaction)


In [ ]:
# evaluating clustering
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(flux_data_T, model_clusters)
print(f'Silhouette Score: {silhouette_avg}')


In [2]:
A375_model = cobra.io.load_matlab_model("./models/A375_ftINIT_model.mat")

In [3]:
A375_model

Name,model
Memory address,25d2c04f440
Number of metabolites,5721
Number of reactions,8577
Number of genes,2413
Number of groups,137
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Extracellular, Cytosol, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [4]:
ihuman = cobra.io.load_matlab_model("./models/Human-GEM.mat")

In [5]:
ihuman

Name,ihuman
Memory address,25d336af380
Number of metabolites,8455
Number of reactions,12971
Number of genes,2887
Number of groups,148
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"
